# Blue Brain BioExplorer
![](../bioexplorer_banner.png)

## Script to create a visualization of the Spike

In [1]:
from bioexplorer import BioExplorer, Protein, Sugars, Quaternion
uri = 'localhost:5000'
be = BioExplorer(uri)
be.reset()
print(be.version())

0.9.0


In [2]:
surface = False
open_spike = False

In [3]:
'''Resources'''
resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
complex_folder = resource_folder + 'pdb/glycans/complex/'

protein_representation = BioExplorer.REPRESENTATION_ATOMS_AND_STICKS
protein_radius_multiplier = 1.0
glycan_representation = BioExplorer.REPRESENTATION_ATOMS_AND_STICKS
glycan_radius_multiplier = 1.0

if surface:
    protein_radius_multiplier = 1.0
    protein_representation = BioExplorer.REPRESENTATION_SURFACE
    glycan_representation = BioExplorer.REPRESENTATION_ATOMS_AND_STICKS

In [4]:
'''Default side view'''
status = be.core_api().set_camera(
    orientation=[0.707, 0, -0.707, 0.0],
    position=[-26.348, 0.172, 0.620],
    target=[-2.728, 0.172, 0.620]
)

In [5]:
'''Protein'''
source = pdb_folder + 'sars-cov-2-v1.pdb'
if open_spike:
    source = pdb_folder + '6vyb.pdb'
    
spike = Protein(
    sources=[source],
    load_hydrogen=False,
    load_non_polymer_chemicals=False,
)

name = be.NAME_PROTEIN_S_CLOSED
if open_spike:
    name = be.NAME_PROTEIN_S_OPEN
status = be.add_protein(
    name=name, protein=spike,
    atom_radius_multiplier=protein_radius_multiplier,
    representation=protein_representation,
)

## Add glycans to protein

In [6]:
glycan_folder = pdb_folder + 'glycans/'
complex_paths = [glycan_folder + 'complex/33.pdb', glycan_folder + 'complex/34.pdb',
                 glycan_folder + 'complex/35.pdb',glycan_folder + 'complex/36.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/24.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/12.pdb']

### High mannose

In [7]:
indices = [61, 122, 234, 603, 709, 717, 801, 1074]

high_mannose_glycans = Sugars(
    rotation=Quaternion(0.707, 0.0, 0.0, 0.707),
    assembly_name=name, name=be.NAME_GLYCAN_HIGH_MANNOSE, 
    protein_name=name, source=high_mannose_paths[0], 
    site_indices=indices,
    representation=glycan_representation,
    atom_radius_multiplier=glycan_radius_multiplier
)
status = be.add_glycans(high_mannose_glycans)

### O-Glycans

In [8]:
protein_name = be.NAME_PROTEIN_S_CLOSED
if open_spike:
    protein_name = be.NAME_PROTEIN_S_OPEN
    
for index in [323, 325]:
    o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index)
    o_glycan = Sugars(
        assembly_name=name, name=o_glycan_name, source=o_glycan_paths[0],
        protein_name=protein_name, site_indices=[index],
        representation=glycan_representation)
    be.add_sugars(o_glycan)

### Complex

In [9]:
indices = [17, 74, 149, 165, 282, 331, 343, 616, 657, 1098, 1134, 1158, 1173, 1194]
if open_spike:
    indices = [17, 74, 149, 165, 282, 331, 343, 657, 1098, 1134, 1158, 1173, 1194]

complex_glycans = Sugars(
    rotation=Quaternion(0.0, 0.0, 0.0, 1.0),
    assembly_name=name, name=be.NAME_GLYCAN_COMPLEX, 
    protein_name=name, source=complex_paths[0], 
    site_indices=indices,
    representation=glycan_representation,
    atom_radius_multiplier=glycan_radius_multiplier
)
status = be.add_glycans(complex_glycans)

## Materials

In [10]:
be.apply_default_color_scheme(
    shading_mode=be.SHADING_MODE_GOODSELL,
    user_parameter=1, specular_exponent=50, glossiness=1)

IntProgress(value=0, max=5)

## Rendering settings

In [11]:
be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=4, max_accum_frames=64)
params = be.core_api().BioExplorerRendererParams()
params.gi_samples = 1
params.gi_weight = 0.4
params.gi_distance = 50
params.shadows = 1.0
params.soft_shadows = 1.0
params.fog_start = 1500
params.fog_thickness = 1500
params.max_bounces = 1
status = be.core_api().set_renderer_params(params)

## Snaphots

In [12]:
output_folder = '/tmp/'

In [13]:
def generate_snapshot(top_view, keywords):

    '''Define path'''
    path = output_folder + 'protein_s'
    if open_spike:
        path += '_open'
    else:
        path += '_closed'
    if surface:
        path += '_surface'
    if top_view:
        path += '_top'
    else:
        path += '_side'
    for keyword in keywords:
        path += '_' + keyword
    path += '.png'
    print('Exporting to ' + path)
        
    '''Camera'''
    if top_view:
        status = be.core_api().set_camera(
            orientation=[0.5, -0.5, 0.5, 0.5],
            position=[0.360, -26.702, 0.346],
            target=[0.360, 0.172, 0.346]    
        )
    else:
        if open_spike:
            status = be.core_api().set_camera(
                orientation=[1.0, 0.0, 0.0, 0.0],
                position=[-1.117, -0.561, -25.184],
                target=[-1.117, -0.561, 0.620]    
            )
        else:
            status = be.core_api().set_camera(
                orientation=[0.707, 0.0, -0.707, 0.0],
                position=[-26.348, 0.172, 0.620],
                target=[-2.728, 0.172, 0.620]
            )
    
    '''Snapshot'''
    from braynsmediamaker import MovieMaker
    mm = MovieMaker(be.core_api())
    mm.create_snapshot(
        path=path, size=[2048,2048], samples_per_pixel=64)

### Functional regions

In [14]:
def apply_functional_regions_color_scheme():
    '''RGB color palette for amino acid indices'''
    l = 0.2
    h = 1.0
    grey = [l,l,l]
    dark_green = [0.0, l, 0.0]
    light_green = [0.0, h, 0.0]
    red = [h, 0.0, 0.0]
    green = [0.0, h, 0.0]
    blue = [0.0, 0.0, h]
    orange = [h, h/2.0, 0.0]
    cyan = [h, 0.0, h]

    region_indices_and_colors = [
        [   1, grey ], [   16, blue], [  306, grey], [  330, green], [438, dark_green], 
        [ 507, green], [  522, grey], [  816, red ], [  835, grey ], [908, orange],
        [ 986, grey ], [ 1076, cyan], [ 1274, grey], [ 2000, grey ]
    ]

    '''Build a palette according to region colors'''
    palette = list()
    for index in range(len(region_indices_and_colors)-1):
        for i in range(region_indices_and_colors[index + 1][0] - 
                       region_indices_and_colors[index][0]):
            palette.append(region_indices_and_colors[index][1])

    '''Apply palette to other chains'''
    status = be.set_protein_color_scheme(
        assembly_name=name, name=name, 
        color_scheme=be.COLOR_SCHEME_CHAINS,
        palette_name='Greys_r', palette_size=5)

    '''Apply palette to region color scheme (optional: only for chain #2)'''
    status = be.set_protein_color_scheme(
        assembly_name=name, name=name, 
        color_scheme=be.COLOR_SCHEME_REGION, palette=palette,
    #     chain_ids=[2]
    )
    
apply_functional_regions_color_scheme()

In [15]:
generate_snapshot(False, ['glycans', 'with_regions'])

Exporting to /tmp/protein_s_closed_side_glycans_with_regions.png


IntProgress(value=0, description='In progress...')

In [16]:
generate_snapshot(True, ['glycans', 'with_regions'])

Exporting to /tmp/protein_s_closed_top_glycans_with_regions.png


IntProgress(value=0, description='In progress...')

### No functional regions

In [17]:
status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_CHAINS,
    palette_name='binary_r', palette_size=10)

In [18]:
generate_snapshot(False, ['glycans'])

Exporting to /tmp/protein_s_closed_side_glycans.png


IntProgress(value=0, description='In progress...')

In [19]:
generate_snapshot(True, ['glycans'])

Exporting to /tmp/protein_s_closed_top_glycans.png


IntProgress(value=0, description='In progress...')

## Brazilian variant

### Modify amino acid sequence

In [20]:
modified_aa_index = 20
status = be.set_protein_amino_acid(
    assembly_name=name, name=name,
    index=modified_aa_index - 1,
    amino_acid_short_name='ASN'
)

indices = [modified_aa_index]
glycan_variant_name = be.NAME_GLYCAN_COMPLEX + '_variant'
complex_glycans = Sugars(
    assembly_name=name, name=glycan_variant_name,
    protein_name=name, source=complex_paths[0], 
    site_indices=indices,
    representation=glycan_representation,
    atom_radius_multiplier=glycan_radius_multiplier
)
status = be.add_glycans(complex_glycans)

In [21]:
for model in be.core_api().scene.models:
    model_id = model['id']
    model_name = model['name']
    if model_name == glycan_variant_name:
        material_ids = list(be.get_material_ids(model_id)['ids'])
        nb_materials = len(material_ids)
        print(model_name)
        palette = list()
        for i in range(nb_materials):
            palette.append([1,0.2,0.2])
        be.set_materials(
            model_ids=[model_id], material_ids=material_ids,
            diffuse_colors=palette, specular_colors=palette
        )        

### Snapshots

In [22]:
apply_functional_regions_color_scheme()

In [23]:
generate_snapshot(False, ['glycans', 'with_regions', 'brazilian_variant_red'])

Exporting to /tmp/protein_s_closed_side_glycans_with_regions_brazilian_variant_red.png


IntProgress(value=0, description='In progress...')

In [24]:
generate_snapshot(True, ['glycans', 'with_regions', 'brazilian_variant_red'])

Exporting to /tmp/protein_s_closed_top_glycans_with_regions_brazilian_variant_red.png


IntProgress(value=0, description='In progress...')

### Restore initial amino acid sequence

In [25]:
modified_aa_index = 20
status = be.set_protein_amino_acid(
    assembly_name=name, name=name,
    index=modified_aa_index - 1,
    amino_acid_short_name='THR'
)

status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_GLYCOSYLATION_SITE,
    palette_name='Set1', palette_size=2)

## Stylish

In [26]:
be.apply_default_color_scheme(
    shading_mode=be.SHADING_MODE_PERLIN,
    user_parameter=0.03, specular_exponent=5, glossiness=0.1)

IntProgress(value=0, max=6)

In [27]:
be.apply_default_color_scheme(
    shading_mode=be.SHADING_MODE_ELECTRON,
    user_parameter=1, glossiness=1.0)

IntProgress(value=0, max=6)

In [28]:
be.apply_default_color_scheme(
    shading_mode=be.SHADING_MODE_GOODSELL,
    user_parameter=3, glossiness=1.0)

IntProgress(value=0, max=6)

## Glycosylation sites

### Sites 20, 331 and 343

In [29]:
ranges = [20, 20, 331, 331, 343, 343]
status = be.set_protein_amino_acid_sequence_as_ranges(
    assembly_name=name, name=name, amino_acid_ranges=ranges)

status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_AMINO_ACID_SEQUENCE,
    palette_name='Set1', palette_size=2)

### Sites 165 and 234

In [30]:
ranges = [165, 165, 234, 234]
status = be.set_protein_amino_acid_sequence_as_ranges(
    assembly_name=name, name=name, amino_acid_ranges=ranges)

status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_AMINO_ACID_SEQUENCE,
    palette_name='Set1', palette_size=2)

## Visualization of electromagnetic fields

In [31]:
from braynsmediamaker import MovieMaker
be = BioExplorer(uri)
core = be.core_api()
mm = MovieMaker(core)

In [32]:
status = be.build_fields(voxel_size=0.01, density=1.0)
fields_model_id = core.scene.models[len(core.scene.models)-1]['id']

In [33]:
image_size = [2160, 2160]
image_samples_per_pixel = 64

output_folder = '/gpfs/bbp.cscs.ch/project/proj117/images/BioExplorer/fields/'
colormap_folder = resource_folder + 'colormap/'

In [34]:
core.set_renderer(
    current='bio_explorer_fields',
    samples_per_pixel=1, subsampling=4, max_accum_frames=image_samples_per_pixel)

params = core.BioExplorerFieldsRendererParams()
params.cutoff = 5000
params.exposure = 3.0
params.alpha_correction = 0.1
params.nb_ray_steps = 16
params.nb_ray_refinement_steps = image_samples_per_pixel
params.use_hardware_randomizer = True
status = core.set_renderer_params(params)

### Transfer function

In [37]:
!pip install ipyTransferFunction

In [38]:
from ipyTransferFunction import TransferFunctionEditor
def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))

def update_palette(tf):
    intensity = 1
    btf = core.get_model_transfer_function(id=fields_model_id)
    colors = list()
    points = list()
    
    nb_points = len(tf.alpha_sliders)
    step = 1.0 / float(nb_points - 1)
    for i in range(nb_points):
        color = hex_to_rgb(tf.color_pickers[i].value)
        colors.append([
            intensity * float(color[0]) / 256.0, 
            intensity * float(color[1]) / 256.0, 
            intensity * float(color[2]) / 256.0])
        points.append([i * step, tf.alpha_sliders[i].value / 255.0])
        
    btf['colormap']['name'] = 'TransferFunctionEditor'
    btf['colormap']['colors'] = colors
    btf['opacity_curve'] = points
    btf['range'] = [tf.data_range[0], tf.data_range[1]]
    core.set_model_transfer_function(id=fields_model_id, transfer_function=btf)

In [39]:
tf = TransferFunctionEditor(
    filename=colormap_folder + 'spike_v1.1dt',
    on_change=update_palette)
tf.set_range((0, 0.008))

In [40]:
tf.set_palette('rainbow_r')

### Snapshots

In [ ]:
status = core.set_camera(
    orientation=[1.0, 0.0, 0.0, 2.83276944882399e-16],
    position=[-4.252900666700188, -3.626166005160136, -24.492872000260288],
    target=[-4.252900666700188, -3.626166005160124, -0.9517030074466356]
)

In [ ]:
mm.create_snapshot(
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder + '/spike_slice.png')